In [1]:
import cv2
import os

train_dir = "/kaggle/input/ucf101-action-recognition/train"
actions = {"Biking","PushUps","JumpingJack","WalkingWithDog","WallPushups"}
output_base_dir = "/kaggle/working/extracted_frames"


def extract_and_resize_frames(video_path, output_dir, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    os.makedirs(output_dir, exist_ok=True)
    
    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, frame_size)  # Resize to 64x64
        frame_path = os.path.join(output_dir, f"frame_{i:04d}.jpg")
        cv2.imwrite(frame_path, resized_frame)
    
    cap.release()
    print(f"Frames resized to {frame_size} and saved to {output_dir}")

# Extract and resize frames from a video
import os

# Function to extract frames for all videos in an action directory
def process_videos_for_actions(actions, train_dir, output_base_dir, frame_size=(64, 64)):
    for action in actions:
        # Path to the current action directory
        action_dir = os.path.join(train_dir, action)
        if not os.path.exists(action_dir):
            print(f"Directory not found for action: {action}")
            continue

        # Output directory for extracted frames of this action
        action_output_dir = os.path.join(output_base_dir, action)
        os.makedirs(action_output_dir, exist_ok=True)

        # Process each video in the action directory
        for video_file in os.listdir(action_dir):
            if video_file.endswith(".avi"):  # Only process .avi files
                video_path = os.path.join(action_dir, video_file)
                video_output_dir = os.path.join(action_output_dir, os.path.splitext(video_file)[0])
                os.makedirs(video_output_dir, exist_ok=True)

                # Extract and resize frames for this video
                print(f"Processing {video_path}...")
                extract_and_resize_frames(video_path, video_output_dir, frame_size)


# Run the function
process_videos_for_actions(actions, train_dir, output_base_dir, frame_size=(64, 64))



Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g03_c03.avi...
Frames resized to (64, 64) and saved to /kaggle/working/extracted_frames/Biking/v_Biking_g03_c03
Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g07_c05.avi...
Frames resized to (64, 64) and saved to /kaggle/working/extracted_frames/Biking/v_Biking_g07_c05
Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g21_c01.avi...
Frames resized to (64, 64) and saved to /kaggle/working/extracted_frames/Biking/v_Biking_g21_c01
Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g17_c01.avi...
Frames resized to (64, 64) and saved to /kaggle/working/extracted_frames/Biking/v_Biking_g17_c01
Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g22_c04.avi...
Frames resized to (64, 64) and saved to /kaggle/working/extracted_frames/Biking/v_Biking_g22_c04
Processing /kaggle/input/ucf101-action-recognition/train/Biking/v_Biking_g0

In [2]:


# Load the data
base_dir = "/kaggle/working/extracted_frames"
actions = {"Biking","PushUps","JumpingJack","WalkingWithDog","WallPushups"}
seq_length = 20



In [3]:
import numpy as np
import os
import cv2

def action_data_generator(base_dir, actions, seq_length=20, frame_size=(64, 64), batch_size=32):
    action_to_label = {action: i for i, action in enumerate(actions)}
    action_iterators = {action: iter(os.listdir(os.path.join(base_dir, action))) for action in actions}

    while True:
        X, y = [], []
        for action in actions:
            action_dir = os.path.join(base_dir, action)

            if not os.path.exists(action_dir):
                print(f"Action directory not found: {action_dir}")
                continue

            # Load videos for the current action
            video_dir_list = os.listdir(action_dir)
            for video_dir in video_dir_list:
                video_path = os.path.join(action_dir, video_dir)
                if os.path.isdir(video_path):
                    frames = []
                    for frame_file in sorted(os.listdir(video_path)):
                        frame_path = os.path.join(video_path, frame_file)
                        frame = cv2.imread(frame_path)
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        frame = cv2.resize(frame, frame_size)
                        frames.append(frame / 255.0)

                    # Create sequences
                    for i in range(len(frames) - seq_length):
                        X.append(frames[i:i + seq_length])
                        y.append(frames[i + 1:i + seq_length + 1])

                        if len(X) == batch_size:
                            yield np.array(X), np.array(y)
                            X, y = [], []
        
        # Yield any remaining samples
        if X:
            yield np.array(X), np.array(y)



In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv3D


def build_predrnn(input_shape):
    """
    Build an RNN for frame prediction.
    """
    model = Sequential([
        ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True, input_shape=input_shape),
        BatchNormalization(),
        ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same', return_sequences=True),
        BatchNormalization(),
        Conv3D(filters=3, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')  # Predict RGB frames
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model








In [5]:
# For batch training
input_shape = (10, 64, 64,3)
model = build_predrnn(input_shape)
data_gen = action_data_generator(base_dir, actions, seq_length=20, batch_size=16)
model.fit(data_gen, steps_per_epoch=100, epochs=50)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


I0000 00:00:1733230576.580908    1925 service.cc:145] XLA service 0x7ec1e005af40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733230576.580980    1925 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1733230583.124833    1925 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


100/100 ━━━━━━━━━━━━━━━━━━━━ 66s 561ms/step - loss: 0.0474 - mae: 0.1566
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0080 - mae: 0.0680
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0178 - mae: 0.1025
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0089 - mae: 0.0718
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0114 - mae: 0.0824
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0107 - mae: 0.0799
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0087 - mae: 0.0737
Epoch 8/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0055 - mae: 0.0582
Epoch 9/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0078 - mae: 0.0671
Epoch 10/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0107 - mae: 0.0803
Epoch 11/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0068 - mae: 0.0665
Epoch 12/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 56s 561ms/step - loss: 0.0056 - mae: 0.0566


In [9]:
model.save("predrnn_model.keras")  # Save the model in the SavedModel format
